In [1]:
from my_import import *
df_train = pd.read_csv('df_train.csv')
df_val = pd.read_csv('df_val.csv')
df_test = pd.read_csv('df_test.csv')
final_cleaned_dataset_df=pd.read_csv('final_cleaned_dataset_df.csv')

#Make sure the genre collumns is in lists not strings
#NEED TO DO THIS EVERYTIME EXPORT DATASET
df_train['genres'] = df_train['genres'].apply(lambda x: list(ast.literal_eval(x)))
df_val['genres'] = df_val['genres'].apply(lambda x: list(ast.literal_eval(x)))
df_test['genres'] = df_test['genres'].apply(lambda x: list(ast.literal_eval(x)))
final_cleaned_dataset_df['genres'] = final_cleaned_dataset_df['genres'].apply(lambda x: list(ast.literal_eval(x)))
df_train

c:\Users\User\anaconda3\envs\nlp_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,title,synopsis,genres,index
0,Touma Kishinden Oni,Shuramaru is hated and feared by the villagers...,[Supernatural],0
1,Sabaku no Kaizoku! Captain Kuppa,"Sometime in the future, the world was complete...","[Action, Adventure]",1
2,On Air Dekinai!,"Set in 2014, the anime follows the adventures ...",[Comedy],2
3,Space Bug,"This story is about Mick, a sleeping chironomi...",[Adventure],11
4,Bakujuu Gasshin Ziguru Hazeru,The anime is based on MegaHouse's line of Zigu...,"[Action, Mecha]",13
...,...,...,...,...
7115,Zero no Tsukaima: Princesses no Rondo Picture ...,Picture Drama episodes included in each DVD vo...,"[Action, Adventure, Comedy, Drama, Fantasy, Ro...",13363
7116,Natsume Yuujinchou Roku Specials,"On a hot summer day, Takashi Natsume and his f...","[Drama, Shoujo, Slice of Life, Supernatural]",13368
7117,Gintama: Nanigoto mo Saiyo ga Kanjin nano de T...,"Due to the arrival of aliens called the ""Amant...","[Action, Comedy, Historical, Mecha, Sci-Fi, Sh...",13372
7118,Naruto: Dai Katsugeki!! Yuki Hime Shinobu Houj...,The Konohagakure Grand Sports Festival has beg...,"[Action, Comedy, Fantasy, Shounen, Sports]",13373


In [2]:
#Top 5 genres in the dataset is Comedy, Action, Other Fantasy and Adventure

#display(final_cleaned_dataset_df)
mlb=MultiLabelBinarizer()
y_true=mlb.fit_transform(final_cleaned_dataset_df['genres'])

#print("Genre Labels:", mlb.classes_)

# Step 1: Find index of 'Comedy'
comedy_index = list(mlb.classes_).index('Comedy')

# Step 2: Create y_pred with all samples tagged as 'Comedy'
# Same shape as y_true, but only 'Comedy' column is 1
y_pred = np.zeros_like(y_true)
y_pred[:, comedy_index] = 1

print_report(y_true,y_pred,"samples avg","f1-score",mlb)
jaccard = jaccard_score(y_true, y_pred, average='samples')
print("Jaccard Similarity:", jaccard)
hr = hit_rate(y_true, y_pred)
print("Hit Rate:", hr)


samples avg f1-score = 0.21680404597960476
Jaccard Similarity: 0.16077615178760857
Hit Rate: 0.4089632708075929


c:\Users\User\anaconda3\envs\nlp_env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [3]:
from sklearn.metrics import f1_score, jaccard_score, precision_score, recall_score, hamming_loss
import numpy as np

# Define top N genres
top_genres = ['Comedy', 'Action', 'Fantasy', 'Adventure', 'Drama']

# Helper: create y_pred from selected top genres
def create_y_pred(genre_list, mlb, y_true):
    indices = [list(mlb.classes_).index(genre) for genre in genre_list]
    y_pred = np.zeros_like(y_true)
    y_pred[:, indices] = 1
    return y_pred

# Helper: calculate metrics
def evaluate(y_true, y_pred):
    f1 = f1_score(y_true, y_pred, average='macro')
    jaccard = jaccard_score(y_true, y_pred, average='macro')
    precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
    recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
    hamming = hamming_loss(y_true, y_pred)
    hit_rate = (np.logical_and(y_true, y_pred).sum(axis=1) > 0).mean()
    return f1, jaccard, precision, recall, hamming, hit_rate

# Loop through Top-1 to Top-5
for n in range(1, 6):
    selected_genres = top_genres[:n]
    y_pred = create_y_pred(selected_genres, mlb, y_true)
    f1, jaccard, precision, recall, hamming, hit_rate = evaluate(y_true, y_pred)
    
    print(f"\nTop-{n} Genres: {selected_genres}")
    print(f"F1 Score:       {f1:.4f}")
    print(f"Jaccard Index:  {jaccard:.4f}")
    print(f"Precision:      {precision:.4f}")
    print(f"Recall:         {recall:.4f}")
    print(f"Hamming Loss:   {hamming:.4f}")
    print(f"Hit Rate:       {hit_rate * 100:.2f}%")



Top-1 Genres: ['Comedy']
F1 Score:       0.0290
Jaccard Index:  0.0204
Precision:      0.0204
Recall:         0.0500
Hamming Loss:   0.1540
Hit Rate:       40.90%

Top-2 Genres: ['Comedy', 'Action']
F1 Score:       0.0528
Jaccard Index:  0.0361
Precision:      0.0361
Recall:         0.1000
Hamming Loss:   0.1728
Hit Rate:       62.68%

Top-3 Genres: ['Comedy', 'Action', 'Fantasy']
F1 Score:       0.0720
Jaccard Index:  0.0479
Precision:      0.0479
Recall:         0.1500
Hamming Loss:   0.1991
Hit Rate:       71.43%

Top-4 Genres: ['Comedy', 'Action', 'Fantasy', 'Adventure']
F1 Score:       0.0899
Jaccard Index:  0.0588
Precision:      0.0588
Recall:         0.2000
Hamming Loss:   0.2273
Hit Rate:       75.11%

Top-5 Genres: ['Comedy', 'Action', 'Fantasy', 'Adventure', 'Drama']
F1 Score:       0.1070
Jaccard Index:  0.0692
Precision:      0.0692
Recall:         0.2500
Hamming Loss:   0.2566
Hit Rate:       83.25%


In [3]:
# Define top genres
top1 = ['Comedy']
top2 = ['Comedy', 'Action']
top3 = ['Comedy', 'Action', 'Other']

# Helper to create y_pred based on selected genres
def create_y_pred(genre_list, mlb, y_true):
    indices = [list(mlb.classes_).index(genre) for genre in genre_list]
    y_pred = np.zeros_like(y_true)
    y_pred[:, indices] = 1
    return y_pred

# Helper to compute metrics
def evaluate(y_true, y_pred):
    f1 = f1_score(y_true, y_pred, average='samples')
    jaccard = jaccard_score(y_true, y_pred, average='samples')
    hit_rate = (np.logical_and(y_true, y_pred).sum(axis=1) > 0).mean()
    return f1, jaccard, hit_rate

# Run evaluations
for label_set, name in zip([top1, top2, top3], ["Top-1", "Top-2", "Top-3"]):
    y_pred = create_y_pred(label_set, mlb, y_true)
    f1, jaccard, hit_rate = evaluate(y_true, y_pred)
    print(f"\n{name} Genres: {label_set}")
    print(f"F1 Score:      {f1:.4f}")
    print(f"Jaccard Index: {jaccard:.4f}")
    print(f"Hit Rate:      {hit_rate*100:.2f}%")


Top-1 Genres: ['Comedy']
F1 Score:      0.1942
Jaccard Index: 0.1417
Hit Rate:      38.62%

Top-2 Genres: ['Comedy', 'Action']
F1 Score:      0.2416
Jaccard Index: 0.1587
Hit Rate:      57.40%

Top-3 Genres: ['Comedy', 'Action', 'Other']
F1 Score:      0.2680
Jaccard Index: 0.1744
Hit Rate:      66.47%


In [6]:
# Define top N genres
top_genres = ['Comedy', 'Action' ,'Fantasy', 'Adventure', 'Drama']

# Helper: create y_pred from selected top genres
def create_y_pred(genre_list, mlb, y_true):
    indices = [list(mlb.classes_).index(genre) for genre in genre_list]
    y_pred = np.zeros_like(y_true)
    y_pred[:, indices] = 1
    return y_pred

# Helper: calculate metrics
def evaluate(y_true, y_pred):
    f1 = f1_score(y_true, y_pred, average='macro')
    jaccard = jaccard_score(y_true, y_pred, average='macro')
    hit_rate = (np.logical_and(y_true, y_pred).sum(axis=1) > 0).mean()
    return f1, jaccard, hit_rate

# Loop through Top-1 to Top-5
for n in range(1, 6):
    selected_genres = top_genres[:n]
    y_pred = create_y_pred(selected_genres, mlb, y_true)
    f1, jaccard, hit_rate = evaluate(y_true, y_pred)
    
    print(f"\nTop-{n} Genres: {selected_genres}")
    print(f"F1 Score:      {f1:.4f}")
    print(f"Jaccard Index: {jaccard:.4f}")
    print(f"Hit Rate:      {hit_rate*100:.2f}%")


Top-1 Genres: ['Comedy']
F1 Score:      0.0290
Jaccard Index: 0.0204
Hit Rate:      40.90%

Top-2 Genres: ['Comedy', 'Action']
F1 Score:      0.0528
Jaccard Index: 0.0361
Hit Rate:      62.68%

Top-3 Genres: ['Comedy', 'Action', 'Fantasy']
F1 Score:      0.0720
Jaccard Index: 0.0479
Hit Rate:      71.43%

Top-4 Genres: ['Comedy', 'Action', 'Fantasy', 'Adventure']
F1 Score:      0.0899
Jaccard Index: 0.0588
Hit Rate:      75.11%

Top-5 Genres: ['Comedy', 'Action', 'Fantasy', 'Adventure', 'Drama']
F1 Score:      0.1070
Jaccard Index: 0.0692
Hit Rate:      83.25%
